In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
from sklearn.neural_network import MLPClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Flatten
from keras.optimizers import SGD

C:\Users\master\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\master\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\master\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
C:\Users\master\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
df = pd.read_csv('../submission files/test.csv')
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
df ['Age'] = df['Age'].fillna(df['Age'].mean())
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
df.isnull().sum()
labels = ['child','adult','old']
df['age_group'] = pd.cut(df['Age'], bins=3, labels=labels)
df['age_group']= df['age_group'].fillna('adult')
labelsfare = ['cheap', 'normal', 'expensive']
df['Fare_group'] = pd.cut(df['Fare'], bins=3, labels=labelsfare)
df['Fare_group']= df['Fare_group'].fillna('cheap')
print(df.isnull().sum())
df

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          326
Embarked         0
age_group        0
Fare_group       0
dtype: int64


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_group,Fare_group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,adult,cheap
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,adult,cheap
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,old,cheap
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,adult,cheap
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,child,cheap
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,23.0,0,0,A.5. 3236,8.0500,NaN,S,child,cheap
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,adult,cheap
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,adult,cheap
416,1308,3,"Ware, Mr. Frederick",male,34.0,0,0,359309,8.0500,NaN,S,adult,cheap


In [4]:
df.drop(columns=['PassengerId', 'Name', 'Cabin', 'Embarked', 'Ticket','Fare', 'Age'], inplace=True)

In [ ]:
df

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
df['IsAlone']= df['IsAlone'].fillna(0)
df.drop(columns=['SibSp', 'Parch', ], inplace=True)
df

In [5]:
X = df.iloc[:,:].values
X

array([[3, 'male', 0, 0, 'adult', 'cheap'],
       [3, 'female', 1, 0, 'adult', 'cheap'],
       [2, 'male', 0, 0, 'old', 'cheap'],
       ...,
       [3, 'male', 0, 0, 'adult', 'cheap'],
       [3, 'male', 0, 0, 'adult', 'cheap'],
       [3, 'male', 1, 1, 'child', 'cheap']], dtype=object)

In [6]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,4,-1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[0.0 1.0 1.0 ... 3 0 0]
 [1.0 0.0 1.0 ... 3 1 0]
 [0.0 1.0 0.0 ... 2 0 0]
 ...
 [0.0 1.0 1.0 ... 3 0 0]
 [0.0 1.0 1.0 ... 3 0 0]
 [0.0 1.0 0.0 ... 3 1 1]]


In [7]:
sc = StandardScaler()
X[:, 2:] = sc.fit_transform(X[:, 2:])
X

array([[0.0, 1.0, 1.004796190585624, ..., 0.8734819050636105,
        -0.4994700165357186, -0.4002477016385638],
       [1.0, 0.0, 1.004796190585624, ..., 0.8734819050636105,
        0.6169923733676523, -0.4002477016385638],
       [0.0, 1.0, -0.9952267030562371, ..., -0.3158191904301653,
        -0.4994700165357186, -0.4002477016385638],
       ...,
       [0.0, 1.0, 1.004796190585624, ..., 0.8734819050636105,
        -0.4994700165357186, -0.4002477016385638],
       [0.0, 1.0, 1.004796190585624, ..., 0.8734819050636105,
        -0.4994700165357186, -0.4002477016385638],
       [0.0, 1.0, -0.9952267030562371, ..., 0.8734819050636105,
        0.6169923733676523, 0.6198958305865561]], dtype=object)

In [13]:
filename='titanic8250.sav'
loaded_modelmlp = pickle.load(open(filename, 'rb'))

In [14]:
predictions= loaded_modelmlp.predict(X).tolist()
print(predictions)

[[0.1993132084608078], [0.7443549633026123], [0.05763012170791626], [0.1993132084608078], [0.5175249576568604], [0.16585776209831238], [0.31193843483924866], [0.06876014173030853], [0.5704846978187561], [0.3121405839920044], [0.1993132084608078], [0.45763924717903137], [0.9867761731147766], [0.11309215426445007], [0.9983487129211426], [0.8484999537467957], [0.12559540569782257], [0.16585776209831238], [0.7443549633026123], [0.31193843483924866], [0.43880999088287354], [0.5857515931129456], [0.9985816478729248], [0.1749678999185562], [0.9751362800598145], [0.06465788185596466], [0.9821552038192749], [0.16585776209831238], [0.45763924717903137], [0.05891658738255501], [0.14277800917625427], [0.20730219781398773], [0.5287472009658813], [0.5287472009658813], [0.45902010798454285], [0.16585776209831238], [0.31193843483924866], [0.5704846978187561], [0.16585776209831238], [0.1993132084608078], [0.17622236907482147], [0.45763924717903137], [0.1993132084608078], [0.9617673754692078], [0.998348

In [ ]:
filename = '../MLPClassifier30854.sav'
loaded_modelmlp = pickle.load(open(filename, 'rb'))

In [15]:
se = pd.Series(predictions)
df['check'] = se
df['check'] = df['check'].str.get(0)
print(se)
finaldf = []
for val in df.check:
    if val >= 0.5:
        finaldf.append(1)
    else:
        finaldf.append(0)

0       [0.1993132084608078]
1       [0.7443549633026123]
2      [0.05763012170791626]
3       [0.1993132084608078]
4       [0.5175249576568604]
               ...          
413    [0.16585776209831238]
414     [0.9985816478729248]
415     [0.1993131786584854]
416     [0.1993131786584854]
417      [0.555677592754364]
Length: 418, dtype: object


In [16]:
sub['Survived'] = finaldf
sub
sub.to_csv('../submission files/readymodel8250.csv', index=False)

In [ ]:
filename = '../models/RandomForestClassifier_model2079.sav'
loaded_modelrfc = pickle.load(open(filename, 'rb'))

In [ ]:
y_predrf = loaded_modelmlp.predict(X)

In [11]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
filename = 'MLPClassifier776.sav'
loaded_modelmlp = pickle.load(open(filename, 'rb'))
y_predrf = loaded_modelmlp.predict(X)

In [ ]:
sub['Survived'] = y_predrf
sub
sub.to_csv('../submission files/readydmlp776.csv', index=False)

In [ ]:
filename = '../models/RandomForestClassifier_model2079.sav'
loaded_modelxgb = pickle.load(open(filename, 'rb'))

In [ ]:
y_predrf = loaded_modelxgb.predict(X)

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
sub['Survived'] = y_predrf
sub
sub.to_csv('../submission files/readyrf2078.csv', index=False)

#  Decision Tree 78

In [ ]:
filename = '../models/XGBClassifier_model2078.sav'
loaded_modeldt = pickle.load(open(filename, 'rb'))


In [ ]:
y_preddt = loaded_modeldt.predict(X)
y_preddt

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub

In [ ]:
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
sub['Survived'] = y_preddt

In [ ]:
sub
sub.to_csv('../submission files/readyxgb2078.csv', index=False)

# KNN

In [ ]:
filename = '../KNN_model.sav'
loaded_modelknn = pickle.load(open(filename, 'rb'))

In [ ]:
y_predknn = loaded_modelknn.predict(X)
y_predknn

# LOGISTIC REGRESSION PREDICTION 76%

In [ ]:
filename = 'LogisticRegression_model.sav'
loaded_modelreg = pickle.load(open(filename, 'rb'))
X

In [ ]:
y_predreg = loaded_modelreg.predict(X)
y_predreg

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub

In [ ]:
sub.drop(columns=['Survived'], inplace=True)


In [ ]:
sub['Survived'] = y_predknn

In [ ]:
sub
sub.to_csv('../submission files/readyknn.csv', index=False)

In [ ]:
filename = 'XGBClassifier_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
X

In [ ]:
xgb  = loaded_model.predict(X)
xgb

In [ ]:
filename = 'DecisionTree_model.sav'
load_model = pickle.load(open(filename, 'rb'))


In [ ]:
dstpredict = load_model.predict(X)

In [ ]:
sub = pd.read_csv('gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)

In [ ]:
sub['Survived'] = xgb

In [ ]:
sub.to_csv('xgbready.csv', index=False)

In [ ]:
filename = 'RandomForest_model.sav'
load_randomf = pickle.load(open(filename, 'rb'))
rf = load_randomf.predict(X)
rf

In [ ]:
filename = '../XGBClassifier80_model.sav'
load_randomxg80 = pickle.load(open(filename, 'rb'))
y_predxgb80 = load_randomxg80.predict(X)

In [ ]:
sub = pd.read_csv('../submission files/gender_submission.csv', delimiter=',')
sub.drop(columns=['Survived'], inplace=True)
sub['Survived'] = y_predxgb80
sub.to_csv('../submission files/readyxgb80.csv', index=False)